In [1]:
import numpy as np

In [2]:
# neural network class definition
class neuralNetwork:
    
    # initialize the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set the number of nodes in each input, hidden and output layer
        self.i_nodes = inputnodes
        self.h_nodes = hiddennodes
        self.o_nodes = outputnodes
        
        # set the learning rate
        self.lr = learningrate

        # link weight matrices wih and who
        # weights inside the arrays are w_i_j, where link is from node i to j in the next layer
        # w11 w21
        # w12 w22 etc

        # the simple way:
        # (we substract 0.5 to have values between -0.5 and +0.5)
        # self.wih = (np.random.rand(self.h_nodes, self.i_nodes) - 0.5)
        # self.who = (np.random.rand(self.o_nodes, self.h_nodes) - 0.5)
        
        # fancier way of doing the above is using a normal distribution around zero
        # with a standard deviation related to the number of incoming links to a node:
        # 1/√(number of incoming links) (in other words: the number of incoming links ** -0.5)
        self.wih = np.random.normal(0.0, pow(self.i_nodes, - 0.5), (self.h_nodes, self.i_nodes))
        self.who = np.random.normal(0.0, pow(self.h_nodes, - 0.5), (self.o_nodes, self.h_nodes))
        
        pass
    
    
    # train the neural network
    def train():
        pass
    
    # query the neural network
    def query():
        pass
    

In [3]:
# number of input, hidden and output nodes
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate
learning_rate = 0.3

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [4]:
print(n.wih)

[[ 0.72979466  1.0147184   0.21148115]
 [ 0.5987324   0.20114175  0.65664306]
 [ 0.31290103 -0.15551066 -0.55417682]]


In [5]:
print(n.who)

[[-0.29524386  1.04582972 -0.12362738]
 [ 0.44837949  0.11291943 -0.29742542]
 [ 0.08103475  0.37431815  0.92439   ]]
